In [1]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
from sklearn.datasets import fetch_20newsgroups

In [3]:
newgroups = fetch_20newsgroups()

In [5]:
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))

In [7]:
from nltk import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\s+', gaps=True)

In [10]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

In [11]:
from string import punctuation
translate = {ord(p):u" " for p in punctuation}

In [13]:
def text_to_tokens(text):
  clean_text = text.lower().translate(translate)
  tokens = [token.strip() for token in tokenizer.tokenize(clean_text) ]
  tokens = [token for token in tokens if token not in stopwords]
  stemmedtokens = [stemmer.stem(token) for token in tokens]
  filteredtokens = [token for token in tokens if len(token) > 2]
  return stemmedtokens

In [14]:
dataset = [text_to_tokens(text) for text in newgroups['data'] ]

In [16]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 44.0 MB/s eta 0:00:00


In [18]:
# dictionary that maps word to unique id's
# prune_At tells how many max unique words can be stored

In [17]:
from gensim.corpora import Dictionary
dictionary = Dictionary(dataset, prune_at=None)

In [ ]:
# remove words if they occur in less than 5 docs, remove if they occur in more than 50% of focs, keep_n to keep only top n words

In [19]:
dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=None)

In [ ]:
# remove unused word indices of deleted words

In [20]:
dictionary.compactify()

In [ ]:
# bag of words

In [21]:
data = [dictionary.doc2bow(doc) for doc in dataset]

In [22]:
from gensim.models import LdaMulticore

In [23]:
topics = 15

In [25]:
lda = LdaMulticore(data, num_topics=topics, id2word=dictionary, passes=10, workers=4, eval_every=None, batch=True, alpha=5/topics, eta=0.01)

In [27]:
!pip install pyLDAvis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 28.0 MB/s eta 0:00:00


In [28]:
import pyLDAvis.gensim_models
import pyLDAvis
import warnings
warnings.filterwarnings('ignore')

In [29]:
pyLDAvis.enable_notebook()

In [30]:
pyLDAvis.gensim_models.prepare(lda, data, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
13    -0.116143  0.060745       1        1  14.769503
1     -0.095461  0.060345       2        1  11.604459
9      0.003220 -0.156160       3        1  10.018071
0      0.472030  0.079582       4        1   9.347432
8      0.051854  0.065605       5        1   7.443826
5     -0.023873 -0.060639       6        1   7.437775
4     -0.060069  0.003740       7        1   6.916242
11    -0.090733 -0.010325       8        1   5.783152
2     -0.075928  0.050731       9        1   5.562119
10    -0.080901  0.061817      10        1   5.428214
14    -0.046562 -0.080595      11        1   4.290181
3     -0.055638  0.000289      12        1   3.294959
12     0.140945 -0.134582      13        1   3.110427
7     -0.064356  0.077351      14        1   2.708953
6      0.041615 -0.017905      15        1   2.284688, topic_info=        Term          Freq         Total Category  logprob  loglift
5682      ax  60423.000000  60423.000000  Default  30.0000  30.0000
278        x   9262.000000   9262.000000  Default  29.0000  29.0000
218        w   5257.000000   5257.000000  Default  28.0000  28.0000
2599     max   4626.000000   4626.000000  Default  27.0000  27.0000
5920       q   6297.000000   6297.000000  Default  26.0000  26.0000
...      ...           ...           ...      ...      ...      ...
153      get    149.535309   6390.009340  Topic15  -5.7533   0.0240
132   comput    135.870073   2926.955667  Topic15  -5.8491   0.7089
232      com    154.524643  11988.394629  Topic15  -5.7204  -0.5724
5682      ax    165.587967  60423.723434  Topic15  -5.6513  -2.1207
270   system    132.195171   4155.047772  Topic15  -5.8765   0.3311

[1526 rows x 6 columns], token_table=       Topic      Freq Term
term                       
496        1  0.006602    0
496        2  0.003982    0
496        3  0.087296    0
496        4  0.384607    0
496        5  0.291128    0
...      ...       ...  ...
10569      4  0.102962   zv
10569      5  0.011440   zv
10569     12  0.011440   zv
10569     13  0.869453   zv
10569     15  0.011440   zv

[8001 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[14, 2, 10, 1, 9, 6, 5, 12, 3, 11, 15, 4, 13, 8, 7])

In [ ]:
# coherence score using c_v metric

In [36]:
from gensim.models import CoherenceModel

model = CoherenceModel(model=lda,texts=dataset, dictionary=dictionary, coherence='c_v')

In [37]:
score = model.get_coherence()

In [38]:
score

np.float64(0.5137454336470901)

In [ ]:
# perplexity score

In [39]:
lda.log_perplexity(data)

np.float64(-7.884634630739686)